follow from https://qiita.com/koshian2/items/01bd9f08444799625607

In [ ]:
from keras.models     import Model
from keras.layers     import Conv2D, Activation, BatchNormalization, Concatenate, AveragePooling2D, Input, GlobalAveragePooling2D, Dense
from keras.optimizers import Adam
from keras.datasets   import cifar10
from keras.utils      import to_categorical
import numpy as np


In [ ]:
# CIFAR-10の読み込み
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print('np.shape(X_train) = (%d, %d, %d, %d)' % np.shape(X_train))
print('np.shape(y_train) = (%d, %d)' % np.shape(y_train))
print('np.shape(X_test)  = (%d, %d, %d, %d)' % np.shape(X_test))
print('np.shape(y_test)  = (%d, %d)' % np.shape(y_test))


In [ ]:
# data value scaling...
X_train         = (X_train.astype('float32') / 255.0)
X_test          = (X_test.astype('float32')  / 255.0)
y_train, y_test = to_categorical(y_train), to_categorical(y_test)


In [ ]:
class DenseNetSimple:
    def __init__(self, 
                 growth_rate, 
                 compression_factor=0.5, 
                 blocks=[1,2,4,3]):
        # growth rate
        self.growth_rate        = growth_rate
        # compression factor
        self.compression_factor = compression_factor
        # create model
        self.model              = self.make_model(blocks)

    # dense block layer
    def dense_block(self, 
                    input_tensor, 
                    input_channels, 
                    num_block):
        # input layer of dense block
        x           = input_tensor
        num_channel = input_channels
        for i in range(num_block):
            # shortcut
            main    = x
            # branch of DenseBlock
            x       = BatchNormalization()(x)
            x       = Activation("relu")(x)
            # bottle-neck 1x1 conv
            x       = Conv2D(32, (1, 1))(x) # x = Conv2D(128, (1, 1))(x)
            x       = BatchNormalization()(x)
            x       = Activation("relu")(x)
            # 3x3 conv (filter num is growth rate)
            x       = Conv2D(self.growth_rate, (3, 3), padding="same")(x)
            # merge shortcut and branch
            x       = Concatenate()([main, x])
            num_channel += self.growth_rate
        return x, num_channel

    # transition layer
    def transition_layer(self, input_tensor, input_channels):
        num_channel = int(input_channels * self.compression_factor)
        # 1x1 conv
        x = Conv2D(num_channel, (1, 1))(input_tensor)
        # average pooling
        x = AveragePooling2D((2, 2))(x)
        return x, num_channel

    # make model
    def make_model(self, blocks):
        # blocks=[6,12,24,16] is same with DenseNet-121
        input = Input(shape=(32,32,3))
        
        # not issue fractional
        n = 4 # n = 16
        
        x = Conv2D(n, (1,1))(input)
        # [dense block] - [transition layer] - [dense block]
        for block_i in range(len(blocks)):
            # transition
            if (block_i != 0):
                x, n = self.transition_layer(x, n)
            # dense block
            x, n = self.dense_block(x, n, blocks[block_i])
        # global average pooling
        x = GlobalAveragePooling2D()(x)
        # output layer
        output = Dense(10, activation="softmax")(x)
        
        # set model
        model = Model(input, output)
        return model


In [ ]:
# k=16の場合
# densenet = DenseNetSimple(16)
densenet = DenseNetSimple(4)
densenet.model.compile(optimizer=Adam(), loss="categorical_crossentropy", metrics=["acc"])
densenet.model.summary()


In [ ]:
# train
history = densenet.model.fit(x=X_train, 
                             y=y_train, 
                             batch_size=64, 
                             epochs=10, 
                             validation_data=(X_test, y_test))


In [ ]:
score = densenet.model.evaluate(x=X_test, 
                                y=y_test, 
                                verbose=0)
print('Test loss     = %.3f' % score[0])
print('Test accuracy = %.3f' % score[1])
